In [43]:
import sys
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from gezi.common import *
from src.config import *
from src.preprocess import *
from src.eval import *
gezi.init_flags()
gezi.set_pandas()

In [44]:
v = 7

In [45]:
V = 7
root = f'../working/offline/{V}/0'
# pairwise two tower, recall model
# pt_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_ext_emlm_mlm.ep1'
# p2t_model = 'all-mpnet-base-v2.flag-pairwise14-2'
pt_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_mlm3'
# pt_model = 'all-mpnet-base-v2.flag-pairwise14-2'
# pt_model2 = 'pmminilm.flag-pairwise14-2-pre_emlm_mlm-mmnilm'
# pt_model2 = 'pmminilm.flag-pairwise14-2-pre_mlm3-pmminilm'
# pt_model2 = 'pmminilm.flag-pairwise14-2'
# pairwise concat, rank model
# pc_model = 'deberta-v3-small.flag-pairwise14-4-cat-insert-extpred-ft.neg_rand_prob-0.neg_strategy-rand-sample'
pc_model = 'deberta-v3-small.flag-pairwise14-4-cat-insert-oof-ft'

# context model
# c_model = 'deberta-v3-small.flag-context4-2-d'
# c_model2 = 'lsg-mminilm.flag-context4-3-d-s-mminilm'
# c_model = 'deberta-v3-small.flag-context4-2-d'
# c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft'
# c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft.list_train_ordered'
c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft.add_probs'
c_model = 'deberta-v3-small.flag-context4-2-d-oof.add_probs'
# c_model = 'deberta-v3-small.flag-context4-2-d-extpred-ft.0804'
# c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft.list_train_ordered'
# c_model2 = 'lsg-mminilm.flag-context4-3-d-s-oof-ft-mminilm'

In [46]:
df_train = pd.read_feather('../working/train.fea')
df_train = df_train[df_train.fold==0]

In [47]:
xp = gezi.load(f'{root}/{pt_model}.eval/valid.pkl')
xc = gezi.load(f'{root}/{c_model}.eval/valid.pkl')
xpc = gezi.load(f'{root}/{pc_model}.eval/valid.pkl')
gezi.sort_dict_byid_(xp, 'cid')
gezi.sort_dict_byid_(xc, 'cid')
gezi.sort_dict_byid_(xpc, 'cid')

In [48]:
# d = pd.DataFrame(gezi.batch2list(xc))
# dg = d.groupby('pred_id')
# max_preds = dg['pred'].max()
# min_preds = dg['pred'].min()
# max_cls_preds = dg['cls_pred'].max()
# min_cls_preds = dg['cls_pred'].min()

In [49]:
xp = gezi.load(f'{root}/{pt_model}.eval/valid.pkl')
# xc = gezi.load(f'{root}/{c_model}.eval/valid.pkl')
xc = gezi.load(f'{root}/{c_model}.eval/valid.pkl')
xpc = gezi.load(f'{root}/{pc_model}.eval/valid.pkl')
gezi.sort_dict_byid_(xp, 'cid')
gezi.sort_dict_byid_(xc, 'cid')
gezi.sort_dict_byid_(xpc, 'cid')

ic(calc_metric(xp, 'cls_pred'))
ic(calc_metric(xc, 'reg_pred'))
ic(calc_metric(xc, 'cls_pred'))
ic(calc_metric(xpc, 'cls_pred'))

[08/10/22 11:31:36] 603994771.py:9 in <module>
                    calc_metric(xp, 'cls_pred'): 0.8940525259130645
[08/10/22 11:31:43] 603994771.py:10 in <module>
                    calc_metric(xc, 'reg_pred'): 0.9133999180447212
[08/10/22 11:31:51] 603994771.py:11 in <module>
                    calc_metric(xc, 'cls_pred'): 0.9139478196978118
[08/10/22 11:31:59] 603994771.py:12 in <module>
                    calc_metric(xpc, 'cls_pred'): 0.9011719210628208


0.9011719210628208

In [50]:
pc_preds = dict(zip(xpc['cid'], xpc['pred']))
pc_cls_preds = dict(zip(xpc['cid'], xpc['cls_pred']))
pc_probs = dict(zip(xpc['cid'], xpc['probs']))

preds = []
cls_preds = []
probs = []
marks = []
for cid, pred, cls_pred, prob in zip(xp['cid'], xp['pred'], xp['cls_pred'], xp['probs']):
  # if prob.max() < 10:
  if prob.max() < 0.8:
    marks.append(1)
    preds.append(pc_preds.get(cid, pred))
    probs.append(pc_probs.get(cid, prob))
    cls_preds.append(pc_cls_preds.get(cid, cls_pred))
  else:
    marks.append(0)
    preds.append(pred)
    probs.append(prob)
    cls_preds.append(cls_pred)
    
xpc = xp.copy()
xpc['pred'] = np.asarray(preds)
xpc['cls_pred'] = np.asarray(cls_preds)
xpc['probs'] = np.asarray(probs)
xpc['mark'] = np.asarray(marks)
ic(xpc['mark'].mean())
ic(calc_metric(xpc, 'cls_pred'))

[08/10/22 11:32:07] 2337548337.py:27 in <module>
                    xpc['mark'].mean(): 0.5019261406823974
[08/10/22 11:32:15] 2337548337.py:28 in <module>
                    calc_metric(xpc, 'cls_pred'): 0.9020743859652581


0.9020743859652581

In [51]:
LABEL_COL = 'rel_rank'
def diff_feats(x, l,r, name):
  if isinstance(l, str):
    x[f'{name}_diff'] = x[l] - x[r]
  else:
    x[f'{name}_diff'] = l - r
  x[f'abs_{name}_diff'] = abs(x[f'{name}_diff'])

def gen_pfeat(x_, topn=None):
  x = {}
  keys = ['pred', 'cls_pred']
  x = {k: x_[k] for k in keys}
  # x['rank_pred'] = x['pred'] * (1 + x_['n_code_cell']) - 0.5
  # x['cls_rank_pred'] = x['cls_pred'] * (1 + x_['n_code_cell']) - 0.5
  x['min_prob'] = x_['probs'].min()
  x['var_prob'] = np.var(x_['probs'])
  if 'sims' in x_:
    x['min_sim'] = x_['sims'].min()
    x['var_sim'] = np.var(x_['sims'])
  
  idxes = (-x_['probs']).argsort()
  # topn = topn or len(idxes)
  topn = topn or 10
  for i in range(topn):
    if i < len(idxes):
      if i > 0:
        x[f'top_pred_{i}'] = (idxes[i] + 0.5) / (x_['n_code_cell'] + 1)
        # x[f'top_rank_{i}'] = idxes[i]
      x[f'top_prob_{i}'] = x_['probs'][idxes[i]]
      if 'sims' in x_:
        x[f'top_sim_{i}'] = x_['sims'][idxes[i]]
    else:
      x[f'top_pred_{i}'] = -1
      # x[f'top_rank_{i}'] = -1
      x[f'top_prob_{i}'] = -1
      if 'sims' in x_:
        x[f'top_sim_{i}'] = -1
  
  diff_feats(x, 'cls_pred', 'pred', 'cls')
  # diff_feats(x, 'cls_rank_pred', 'rank_pred', 'cls_rank')
  return x

def gen_cfeat(x_):
  topn = min(10, len(x_['cls_pred_ori']))
  x = {}
  keys = [
    # 'pred', 
    'cls_pred', 
    'reg_pred', 
    'cls2_pred', 
    'group_pos',
    'group_rank',
    ]
  x = {k: x_[k] for k in keys}
  # x['cls_rank_pred'] = x['cls_pred'] * FLAGS.num_classes - 0.5
  # x['cls2_rank_pred'] = x['cls2_pred'] * FLAGS.num_classes - 0.5
  # x['reg_rank_pred'] = x['reg_pred'] * FLAGS.num_classes - 0.5
  preds = x_['cls_pred_ori']
  probs = gezi.softmax(preds)
  x['min_prob'] = probs.min()
  x['var_prob'] = np.var(probs)
  idxes = (-probs).argsort()

  for i in range(topn):
    if i < len(idxes):
      if i > 0:
        x[f'top_pred_{i}'] = (idxes[i] + 0.5) / FLAGS.num_classes
        # x[f'tpop_rank_{i}']  = idxes[i] 
      x[f'top_prob_{i}'] = probs[idxes[i]]
    else:
      x[f'top_pred_{i}'] = -1
      # x[f'top_rank_{i}'] = -1
      x[f'top_prob_{i}'] = -1
  diff_feats(x, 'cls_pred', 'reg_pred', 'cls_reg')
  diff_feats(x, 'cls_pred', 'cls2_pred', 'cls_cls2')
  diff_feats(x, 'cls2_pred', 'reg_pred', 'cls2_reg')
  # diff_feats(x, 'cls_rank_pred', 'reg_rank_pred', 'cls_reg_rank')
  # diff_feats(x, 'cls_rank_pred', 'cls2_rank_pred', 'cls_cls2_rank')
  # diff_feats(x, 'cls2_rank_pred', 'reg_rank_pred', 'cls2_reg_rank')
  # group_id = x_['pred_id']
  # x['max_pred'] = max_preds[group_id]
  # x['min_pred'] = min_preds[group_id]
  # x['pred_ratio'] = (x['pred'] - x['min_pred']) / (x['max_pred'] - x['min_pred']) if (x['max_pred'] - x['min_pred']) else 1
  # x['max_cls_pred'] = max_cls_preds[group_id]
  # x['min_cls_pred'] = min_cls_preds[group_id]
  # x['cls_pred_ratio'] = (x['cls_pred'] - x['min_cls_pred']) / (x['max_cls_pred'] - x['min_cls_pred']) if (x['max_cls_pred'] - x['min_cls_pred']) else 1
  
  return x

def gen_feats():
  xs = gezi.batch2list(xp)
  p_feats = [gen_pfeat(x) for x in tqdm(xs, desc=f'gen_pfeats:xp')]
  xs = gezi.batch2list(xc)
  c_feats = [gen_cfeat(x) for x in tqdm(xs, desc=f'gen_cfeats:xc')]
  xs = gezi.batch2list(xpc)
  pc_feats = [gen_pfeat(x, topn=3) for x in tqdm(xs, desc=f'gen_pfeats:xpc')]

  feats = []
  for i in tqdm(range(len(p_feats)), desc='merge_feats'):
    fe1 = p_feats[i]
    fec = c_feats[i]
    fepc = pc_feats[i]
    fepc['mark'] = xpc['mark'][i]
    
    fe = {}
    fe['code_ratio'] = xp['n_code_cell'][i] / xp['n_cell'][i]
    diff_feats(fe, fe1['pred'], fec['reg_pred'], 'pc_reg')
    diff_feats(fe, fe1['pred'], fec['cls_pred'], 'pc_cls')
    diff_feats(fe, fe1['pred'], fec['cls2_pred'], 'pc_cls2')
    diff_feats(fe, fe1['cls_pred'], fec['reg_pred'], 'pc_cls_reg')
    diff_feats(fe, fe1['cls_pred'], fec['cls_pred'], 'pc_cls_cls')
    diff_feats(fe, fe1['cls_pred'], fec['cls2_pred'], 'pc_cls_cls2')
    
    diff_feats(fe, fepc['pred'], fec['reg_pred'], 'pcc_reg')
    diff_feats(fe, fepc['pred'], fec['cls_pred'], 'pcc_cls')
    diff_feats(fe, fepc['pred'], fec['cls2_pred'], 'pcc_cls2')
    diff_feats(fe, fepc['cls_pred'], fec['reg_pred'], 'pcc_cls_reg')
    diff_feats(fe, fepc['cls_pred'], fec['cls_pred'], 'pcc_cls_cls')
    diff_feats(fe, fepc['cls_pred'], fec['cls2_pred'], 'pcc_cls_cls2')
    
    # diff_feats(fe, fe1['rank_pred'], fec['reg_rank_pred'], 'pc_cls_rank')
    # diff_feats(fe, fe1['cls_rank_pred'], fec['reg_rank_pred'], 'pc_cls_reg_rank')
    # diff_feats(fe, fepc['rank_pred'], fec['reg_rank_pred'], 'pcc_cls_rank')
    # diff_feats(fe, fepc['cls_rank_pred'], fec['reg_rank_pred'], 'pcc_cls_reg_rank')

    fe1 = gezi.dict_prefix(fe1, 'xp_')
    fe.update(fe1)
    fec = gezi.dict_prefix(fec, 'xc_')
    fe.update(fec)
    fepc = gezi.dict_prefix(fepc, 'xpc_')
    fe.update(fepc)

    keys = [
     'id', 'cell_id', 'cid',
     'n_words', 'n_code_cell', 'n_cell'
    ]
    for key in keys:
      fe[key] = xp[key][i]
      
    #  ic(feat)
    feats.append(fe)
  #  break
  dfeats = pd.DataFrame(feats)
  return dfeats

In [52]:
dfeats = gen_feats()

gen_pfeats:xp:   0%|          | 0/424943 [00:00<?, ?it/s]

gen_cfeats:xc:   0%|          | 0/424943 [00:00<?, ?it/s]

gen_pfeats:xpc:   0%|          | 0/424943 [00:00<?, ?it/s]

merge_feats:   0%|          | 0/424943 [00:00<?, ?it/s]

In [53]:
df = dfeats.merge(df_train[['cid', 'ancestor_id', LABEL_COL]], on='cid')

In [54]:
gezi.set_fold(df, 5, 'ancestor_id')

In [55]:
reg_cols = [x for x in dfeats.columns if x not in ['id', 'cell_id', 'cid', LABEL_COL]]
cat_cols = []
cols = reg_cols + cat_cols
len(cols)

114

In [56]:
df['cls_label'] = df.rel_rank.apply(lambda x: int(x * 100))
LABEL_COL = 'rel_rank'

In [57]:
df.cls_label.max()

99

In [58]:
fold = 0
dvalid = df[df.fold==fold]
dtrain = df[df.fold!=fold]

In [59]:
X_train = dtrain[cols]
y_train = dtrain[LABEL_COL]
X_valid = dvalid[cols]
y_valid = dvalid[LABEL_COL]

In [60]:
from catboost import CatBoostRegressor, CatBoostRanker, CatBoostClassifier
cbt_params = {
              # 'bootstrap_type': 'Poisson',
              # 'learning_rate': 0.02,
              'learning_rate': 0.03,
              # 'reg_lambda': 7.960622217848342e-07, 
              'l2_leaf_reg': 3,
              'subsample': 0.7422597612762745,
              # 'bagging_temperature': 0.2,
              # 'max_depth': 10, 
              # 'max_depth': 8,
              'max_depth': 6,
              # 'max_depth': 5,
              'early_stopping_rounds': 500,
              'n_estimators': 10000,
              'cat_features': [],
              'loss_function': 'MAE',
              'min_child_samples': 100,
              # 'task_type': 'GPU',
              # 'devices': '0',
              }

In [61]:
cbt_model = CatBoostRegressor(**cbt_params)
cbt_model.fit(X_train, y_train,
      eval_set=[(X_train, y_train), (X_valid, y_valid)],
              verbose=100,
              )  
dvalid['cbt_pred'] = cbt_model.predict(dvalid[cols])
cbt_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cbt_pred.values})
ic(cbt_score)

0:	learn: 0.2523411	test: 0.2523411	test1: 0.2522854	best: 0.2522854 (0)	total: 44ms	remaining: 7m 20s
100:	learn: 0.0658321	test: 0.0658321	test1: 0.0670874	best: 0.0670874 (100)	total: 3.79s	remaining: 6m 11s
200:	learn: 0.0590696	test: 0.0590696	test1: 0.0605187	best: 0.0605187 (200)	total: 7.34s	remaining: 5m 57s
300:	learn: 0.0582404	test: 0.0582404	test1: 0.0597643	best: 0.0597643 (300)	total: 10.9s	remaining: 5m 49s
400:	learn: 0.0576440	test: 0.0576440	test1: 0.0592171	best: 0.0592171 (400)	total: 14.5s	remaining: 5m 47s
500:	learn: 0.0570862	test: 0.0570862	test1: 0.0587003	best: 0.0587003 (500)	total: 18.3s	remaining: 5m 46s
600:	learn: 0.0567262	test: 0.0567262	test1: 0.0583919	best: 0.0583919 (600)	total: 22s	remaining: 5m 44s
700:	learn: 0.0564415	test: 0.0564415	test1: 0.0581537	best: 0.0581537 (700)	total: 25.8s	remaining: 5m 42s
800:	learn: 0.0562346	test: 0.0562346	test1: 0.0579898	best: 0.0579898 (800)	total: 29.6s	remaining: 5m 40s
900:	learn: 0.0560743	test: 0.05607

[08/10/22 11:38:41] 3198672829.py:8 in <module>
                    cbt_score: 0.9194913580046135


0.9194913580046135

In [62]:
dvalid

,code_ratio,pc_reg_diff,abs_pc_reg_diff,pc_cls_diff,abs_pc_cls_diff,pc_cls2_diff,abs_pc_cls2_diff,pc_cls_reg_diff,abs_pc_cls_reg_diff,pc_cls_cls_diff,abs_pc_cls_cls_diff,pc_cls_cls2_diff,abs_pc_cls_cls2_diff,pcc_reg_diff,abs_pcc_reg_diff,pcc_cls_diff,abs_pcc_cls_diff,pcc_cls2_diff,abs_pcc_cls2_diff,pcc_cls_reg_diff,abs_pcc_cls_reg_diff,pcc_cls_cls_diff,abs_pcc_cls_cls_diff,pcc_cls_cls2_diff,abs_pcc_cls_cls2_diff,xp_pred,xp_cls_pred,xp_min_prob,xp_var_prob,xp_min_sim,xp_var_sim,xp_top_prob_0,xp_top_sim_0,xp_top_pred_1,xp_top_prob_1,xp_top_sim_1,xp_top_pred_2,xp_top_prob_2,xp_top_sim_2,xp_top_pred_3,xp_top_prob_3,xp_top_sim_3,xp_top_pred_4,xp_top_prob_4,xp_top_sim_4,xp_top_pred_5,xp_top_prob_5,xp_top_sim_5,xp_top_pred_6,xp_top_prob_6,xp_top_sim_6,xp_top_pred_7,xp_top_prob_7,xp_top_sim_7,xp_top_pred_8,xp_top_prob_8,xp_top_sim_8,xp_top_pred_9,xp_top_prob_9,xp_top_sim_9,xp_cls_diff,xp_abs_cls_diff,xc_cls_pred,xc_reg_pred,xc_cls2_pred,xc_group_pos,xc_group_rank,xc_min_prob,xc_var_prob,xc_top_prob_0,xc_top_pred_1,xc_top_prob_1,xc_top_pred_2,xc_top_prob_2,xc_top_pred_3,xc_top_prob_3,xc_top_pred_4,xc_top_prob_4,xc_top_pred_5,xc_top_prob_5,xc_top_pred_6,xc_top_prob_6,xc_top_pred_7,xc_top_prob_7,xc_top_pred_8,xc_top_prob_8,xc_top_pred_9,xc_top_prob_9,xc_cls_reg_diff,xc_abs_cls_reg_diff,xc_cls_cls2_diff,xc_abs_cls_cls2_diff,xc_cls2_reg_diff,xc_abs_cls2_reg_diff,xpc_pred,xpc_cls_pred,xpc_min_prob,xpc_var_prob,xpc_min_sim,xpc_var_sim,xpc_top_prob_0,xpc_top_sim_0,xpc_top_pred_1,xpc_top_prob_1,xpc_top_sim_1,xpc_top_pred_2,xpc_top_prob_2,xpc_top_sim_2,xpc_cls_diff,xpc_abs_cls_diff,xpc_mark,id,cell_id,cid,n_words,n_code_cell,n_cell,ancestor_id,rel_rank,fold,cls_label,cbt_pred
54,0.5895,-0.0033,0.0033,-0.0015,0.0015,0.0018,0.0018,-0.0010,0.0010,0.0008,0.0008,0.0041,0.0041,-0.0033,0.0033,-0.0015,0.0015,0.0018,0.0018,-0.0010,0.0010,0.0008,0.0008,0.0041,0.0041,0.2368,0.2391,0.0000,0.0160,0.3837,0.0085,0.9640,0.7873,0.2544,0.0204,0.7026,0.5351,0.0038,0.6660,0.3070,0.0016,0.6462,0.3772,0.0015,0.6456,0.1491,0.0014,0.6436,0.4825,0.0014,0.6435,0.1667,0.0010,0.6366,0.0263,0.0008,0.6302,0.2895,0.0006,0.6243,0.0023,0.0023,0.2383,0.2401,0.2350,1,1,0.0000,0.0079,0.8896,0.2550,0.0975,0.2850,0.0018,0.2150,0.0017,0.2450,0.0016,0.2650,0.0009,0.1550,0.0008,0.2750,0.0008,0.4150,0.0007,0.3350,0.0007,-0.0018,0.0018,0.0033,0.0033,-0.0051,0.0051,0.2368,0.2391,0.0000,0.0160,0.3837,0.0085,0.9640,0.7873,0.2544,0.0204,0.7026,0.5351,0.0038,0.6660,0.0023,0.0023,0,001308991e0c5e,05915708,001308991e0c5e\t05915708,22,56,95,c5fbea8e,0.2368,0,23,0.2370
55,0.5895,-0.0113,0.0113,-0.0116,0.0116,-0.0018,0.0018,-0.0091,0.0091,-0.0094,0.0094,0.0003,0.0003,-0.0113,0.0113,-0.0116,0.0116,-0.0018,0.0018,-0.0091,0.0091,-0.0094,0.0094,0.0003,0.0003,0.7632,0.7653,0.0000,0.0134,0.3038,0.0060,0.8777,0.7252,0.7807,0.1110,0.6798,0.7982,0.0098,0.6266,0.8333,0.0004,0.5554,0.8684,0.0003,0.5486,0.8158,0.0002,0.5427,0.6053,0.0001,0.5321,0.0263,0.0001,0.5271,0.5877,0.0001,0.5164,0.8509,0.0000,0.5046,0.0022,0.0022,0.7747,0.7744,0.7650,1,1,0.0000,0.0047,0.5229,0.7850,0.4507,0.7750,0.0114,0.7950,0.0071,0.7550,0.0036,0.7450,0.0019,0.7350,0.0004,0.8150,0.0003,0.7250,0.0003,0.7050,0.0002,0.0003,0.0003,0.0097,0.0097,-0.0094,0.0094,0.7632,0.7653,0.0000,0.0134,0.3038,0.0060,0.8777,0.7252,0.7807,0.1110,0.6798,0.7982,0.0098,0.6266,0.0022,0.0022,0,001308991e0c5e,1545964f,001308991e0c5e\t1545964f,22,56,95,c5fbea8e,0.7632,0,76,0.7666
56,0.5895,-0.0018,0.0018,0.0036,0.0036,0.0048,0.0048,-0.0020,0.0020,0.0034,0.0034,0.0046,0.0046,-0.0018,0.0018,0.0036,0.0036,0.0048,0.0048,-0.0020,0.0020,0.0034,0.0034,0.0046,0.0046,0.4298,0.4296,0.0000,0.0171,0.2332,0.0078,0.9970,0.6821,0.3070,0.0018,0.5436,0.3246,0.0003,0.5028,0.4825,0.0003,0.5013,0.5175,0.0002,0.4981,0.5000,0.0002,0.4900,0.4474,0.0001,0.4858,0.3772,0.0000,0.4596,0.4649,0.0000,0.4512,0.3421,0.0000,0.4239,-0.0002,0.0002,0.4262,0.4316,0.4250,2,1,0.0000,0.0061,0.7432,0.4350,0.2489,0.4150,0.0020,0.4450,0.0008,0.4750,0.0003,0.4550,0.0002,0.4050,0.0002,0.4650,0.0002,0.3050

In [63]:
gezi.plot.feature_importance(cbt_model, topn=20)

In [64]:
import lightgbm as lgb
params = {
          'boosting': 'gbdt',
          'objective': 'regression_l1',
          'metric': {'l1'},
          # 'num_leaves': 32,
          # 'num_leaves': 24,
          'num_leaves': 12,
          # 'min_data_in_leaf': 300,
          'min_data_in_leaf': 500,
          # 'max_depth': 5,
          'learning_rate': 0.03,
          # "feature_fraction": 0.8,
          # "bagging_fraction": 0.75,
          # "feature_fraction": 0.9,
          # "bagging_fraction": 0.8,
          'feature_fraction': 0.7,
          'bagging_fraction': 0.7,
          'min_data_in_bin':100,
          "lambda_l1": 1e-7,
          'lambda_l2': 1e-7,
          "random_state": 1024,
          "num_threads": 12,
          }

In [65]:
d_train = lgb.Dataset(X_train, y_train)
d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)

lgb_model = lgb.train(params,
                d_train,
                20000,
                valid_sets=[d_train, d_valid],
                verbose_eval=100,
                early_stopping_rounds=200)
dvalid['lgb_pred'] = lgb_model.predict(dvalid[cols])
lgb_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})
ic(lgb_score)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.057236 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26661
[LightGBM] [Info] Number of data points in the train set: 342411, number of used features: 114
[LightGBM] [Info] Start training from score 0.455056
Training until validation scores don't improve for 200 rounds
[100]	training's l1: 0.0668261	valid_1's l1: 0.0680372
[200]	training's l1: 0.0595717	valid_1's l1: 0.0609744
[300]	training's l1: 0.058644	valid_1's l1: 0.0600934
[400]	training's l1: 0.057801	valid_1's l1: 0.0593145
[500]	training's l1: 0.0571562	valid_1's l1: 0.0587155
[600]	training's l1: 0.0568031	valid_1's l1: 0.0583954
[700]	training's l1: 0.0565955	valid_1's l1: 0.0582078
[800]	training's l1: 0.0563526	valid_1's l1: 0.0580062
[900]	training's l1: 0.0561984	valid_1's l1: 0.0578756
[1000]	training's l1: 0.0560919	valid_1's l1: 0.0577847
[1100]	training's l1: 0.0559949	valid_1's

[08/10/22 11:42:41] 1296901836.py:12 in <module>
                    lgb_score: 0.9197838846981955


0.9197838846981955

In [66]:
gezi.plot.feature_importance(lgb_model, topn=20)

In [67]:
FOLDS = 5
cbt_scores, lgb_scores = [], []
scores = []
gezi.try_mkdir(f'../working/trees{v}')
for fold in tqdm(range(FOLDS)):
  dvalid = df[df.fold==fold]
  dtrain = df[df.fold!=fold]
  X_train = dtrain[cols]
  y_train = dtrain[LABEL_COL]
  X_valid = dvalid[cols]
  y_valid = dvalid[LABEL_COL]
  if fold > 0:
    cbt_model = CatBoostRegressor(**cbt_params)
    cbt_model.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid, y_valid)],
                  verbose=500,
                  )  
  cbt_model.save_model(f'../working/trees{v}/{fold}.cbt')
  dvalid['cbt_pred'] = cbt_model.predict(dvalid[cols])
  cbt_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cbt_pred.values})
  ic(cbt_score)
  
  d_train = lgb.Dataset(X_train, y_train)
  d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)
  if fold > 0:
    lgb_model = lgb.train(params,
                    d_train,
                    20000,
                    valid_sets=[d_train, d_valid],
                    verbose_eval=500,
                    early_stopping_rounds=200)
  lgb_model.save_model(f'../working/trees{v}/{fold}.lgb')
  dvalid['lgb_pred'] = lgb_model.predict(dvalid[cols])
  
  lgb_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})
  ic(lgb_score)
  score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': (dvalid.cbt_pred.values * 0.5 + dvalid.lgb_pred.values * 0.5)})
  ic(score)
  
  cbt_scores.append(cbt_score)
  lgb_scores.append(lgb_score)
  scores.append(score)
  ic(fold, 
     np.asarray(cbt_scores).mean(), 
     np.asarray(lgb_scores).mean(),
     np.asarray(scores).mean(),
     )

  0%|          | 0/5 [00:00<?, ?it/s]

[08/10/22 11:42:51] 134356872.py:21 in <module>
                    cbt_score: 0.9194913580046135
[08/10/22 11:43:18] 134356872.py:36 in <module>
                    lgb_score: 0.9197838846981955
[08/10/22 11:43:19] 134356872.py:38 in <module>
                    score: 0.9199306692971134
[08/10/22 11:43:19] 134356872.py:43 in <module>
                    fold: 0
                    np.asarray(cbt_scores).mean(): 0.9194913580046135
                    np.asarray(lgb_scores).mean(): 0.9197838846981955
                    np.asarray(scores).mean(): 0.9199306692971134


0:	learn: 0.2522810	test: 0.2522810	test1: 0.2521630	best: 0.2521630 (0)	total: 49.1ms	remaining: 8m 10s
500:	learn: 0.0576572	test: 0.0576572	test1: 0.0569063	best: 0.0569063 (500)	total: 17.8s	remaining: 5m 38s
1000:	learn: 0.0564196	test: 0.0564196	test1: 0.0558777	best: 0.0558777 (1000)	total: 36.1s	remaining: 5m 24s
1500:	learn: 0.0559198	test: 0.0559198	test1: 0.0555497	best: 0.0555497 (1500)	total: 54.4s	remaining: 5m 8s
2000:	learn: 0.0555636	test: 0.0555636	test1: 0.0553806	best: 0.0553806 (2000)	total: 1m 12s	remaining: 4m 50s
2500:	learn: 0.0552639	test: 0.0552639	test1: 0.0552837	best: 0.0552837 (2500)	total: 1m 30s	remaining: 4m 32s
3000:	learn: 0.0549859	test: 0.0549859	test1: 0.0552047	best: 0.0552047 (3000)	total: 1m 48s	remaining: 4m 13s
3500:	learn: 0.0547591	test: 0.0547591	test1: 0.0551500	best: 0.0551495 (3492)	total: 2m 6s	remaining: 3m 55s
4000:	learn: 0.0545415	test: 0.0545415	test1: 0.0551049	best: 0.0551019 (3977)	total: 2m 24s	remaining: 3m 37s
4500:	learn: 0

[08/10/22 11:48:26] 134356872.py:21 in <module>
                    cbt_score: 0.9225357904559981


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.058962 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26660
[LightGBM] [Info] Number of data points in the train set: 339696, number of used features: 114
[LightGBM] [Info] Start training from score 0.455882
Training until validation scores don't improve for 200 rounds
[500]	training's l1: 0.0576642	valid_1's l1: 0.0568263
[1000]	training's l1: 0.056601	valid_1's l1: 0.0559152
[1500]	training's l1: 0.0561799	valid_1's l1: 0.0555909
[2000]	training's l1: 0.0559751	valid_1's l1: 0.0554544
[2500]	training's l1: 0.0558391	valid_1's l1: 0.0553679
[3000]	training's l1: 0.0556948	valid_1's l1: 0.0552878
[3500]	training's l1: 0.0555545	valid_1's l1: 0.0552168
[4000]	training's l1: 0.0554536	valid_1's l1: 0.0551797
[4500]	training's l1: 0.0553673	valid_1's l1: 0.0551497
[5000]	training's l1: 0.0552714	valid_1's l1: 0.0551134
[5500]	training's l1: 0.055191	v

...

[08/10/22 12:05:34] 134356872.py:36 in <module>
                    lgb_score: 0.9242514920118299
[08/10/22 12:05:36] 134356872.py:38 in <module>
                    score: 0.9247517138237571
[08/10/22 12:05:36] 134356872.py:43 in <module>
                    fold: 3
                    np.asarray(cbt_scores).mean(): 0.9220723247484163
                    np.asarray(lgb_scores).mean(): 0.9220011472532595
                    np.asarray(scores).mean(): 0.9223373810393756


0:	learn: 0.2523147	test: 0.2523147	test1: 0.2524000	best: 0.2524000 (0)	total: 48.6ms	remaining: 8m 5s
500:	learn: 0.0570829	test: 0.0570829	test1: 0.0583990	best: 0.0583990 (500)	total: 18.5s	remaining: 5m 50s
1000:	learn: 0.0560056	test: 0.0560056	test1: 0.0574860	best: 0.0574860 (1000)	total: 37.5s	remaining: 5m 37s
1500:	learn: 0.0555433	test: 0.0555433	test1: 0.0571625	best: 0.0571625 (1500)	total: 56.6s	remaining: 5m 20s
2000:	learn: 0.0552028	test: 0.0552028	test1: 0.0569758	best: 0.0569758 (2000)	total: 1m 15s	remaining: 5m 1s
2500:	learn: 0.0549101	test: 0.0549101	test1: 0.0568543	best: 0.0568543 (2500)	total: 1m 34s	remaining: 4m 43s
3000:	learn: 0.0546693	test: 0.0546693	test1: 0.0567606	best: 0.0567606 (3000)	total: 1m 59s	remaining: 4m 38s
3500:	learn: 0.0544379	test: 0.0544379	test1: 0.0567115	best: 0.0567114 (3496)	total: 2m 23s	remaining: 4m 26s
4000:	learn: 0.0542311	test: 0.0542311	test1: 0.0566669	best: 0.0566669 (4000)	total: 2m 47s	remaining: 4m 11s
4500:	learn: 0

[08/10/22 12:10:55] 134356872.py:21 in <module>
                    cbt_score: 0.9154773103522206


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.059305 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26659
[LightGBM] [Info] Number of data points in the train set: 341935, number of used features: 114
[LightGBM] [Info] Start training from score 0.456522
Training until validation scores don't improve for 200 rounds
[500]	training's l1: 0.0573576	valid_1's l1: 0.0586868
[1000]	training's l1: 0.0561203	valid_1's l1: 0.0575597
[1500]	training's l1: 0.055775	valid_1's l1: 0.0573012
[2000]	training's l1: 0.0555734	valid_1's l1: 0.0571567
[2500]	training's l1: 0.0554168	valid_1's l1: 0.0570568
[3000]	training's l1: 0.0552992	valid_1's l1: 0.0569831
[3500]	training's l1: 0.055218	valid_1's l1: 0.056944
[4000]	training's l1: 0.0551415	valid_1's l1: 0.0569054
[4500]	training's l1: 0.0550616	valid_1's l1: 0.0568712
[5000]	training's l1: 0.0549678	valid_1's l1: 0.056834
[5500]	training's l1: 0.0549044	val

[08/10/22 12:15:43] 134356872.py:36 in <module>
                    lgb_score: 0.9153345251807234
[08/10/22 12:15:44] 134356872.py:38 in <module>
                    score: 0.9156358632113677
[08/10/22 12:15:45] 134356872.py:43 in <module>
                    fold: 4
                    np.asarray(cbt_scores).mean(): 0.9207533218691772
                    np.asarray(lgb_scores).mean(): 0.9206678228387524
                    np.asarray(scores).mean(): 0.9209970774737741


In [68]:
# fold: 0
#                     np.asarray(cbt_scores).mean(): 0.9181025338378637
#                     np.asarray(lgb_scores).mean(): 0.9193848140698876
#                     np.asarray(scores).mean(): 0.9195238841373282

#  fold: 4
#                     np.asarray(cbt_scores).mean(): 0.9189608630607822
#                     np.asarray(lgb_scores).mean(): 0.9196959050732708
#                     np.asarray(scores).mean(): 0.9199400102856614